In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pylab as py
import pandas as pd
import requests
import urllib
import urllib.request
import html.parser
import requests
from selenium import webdriver
driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")

In [2]:
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

In [3]:
# match indexing for EPL url
i_18_19 = [38308, 38687]
i_17_18 = [22342, 22721]
i_16_17 = [14040, 14419]
i_15_16 = [12115, 12494]
i_14_15 = [9611, 9990]
i_13_14 = [9231, 9610]
i_12_13 = [7864, 8243]
i_11_12 = [7467, 7846]

In [15]:
# import html from url
print('yes')
url = "https://www.premierleague.com/match/38678"
#stats_url = 'https://datahub.io/sports-data/english-premier-league/datapackage.json'
team_url='https://www.whoscored.com/Teams/26/Archive/England-Liverpool'
base_url='https://www.whoscored.com'

source_code = requests.get(url)
team_text=driver.get(team_url)
#print(team_text)


soup_team = BeautifulSoup(driver.page_source, 'lxml')
#print(soup_team)
'''
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)
'''
plain_text = source_code.text
soup = BeautifulSoup(plain_text, 'html.parser')
#print(soup)




yes


In [5]:
# generate data_ppp: data from EPL website containing (p)layer name, (p)osition, and (p)layed time
role = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
data_ppp = soup.find_all('div', class_='matchLineupTeamContainer')
match_stats=soup.find_all('div', class_='matchLineupTeamContainer')
#print((data_ppp))




In [6]:
M_ppp = [[], []] # matrix for home & away 'ppp' information
for team_num in range(2): # iterate over home & away teams
    team_i = data_ppp[team_num].getText().split('\n')
    team_i = [element.strip() for element in team_i if (element.strip() != '' and element.strip() != 'C')]
    # team_i now is a list: [..."p_i uniform #", "p_i name", "p_i position", "##'; subbed in/out time",
    #                        ... "Substitues", "p_sub_i uniform #", ...]
    i_sub = team_i.index('Substitutes') # index showing 'Substitues' header
    team_i_11, team_i_sub = team_i[:i_sub], team_i[i_sub+1:] # distinguish between starting 11 and sub's
    # iterate over starting 11 players
    i_role = [0] + [i for i in range(len(team_i_11)) if team_i_11[i] in role]
    for i in range(len(i_role)-1):
        if i == 0: L_i = team_i_11[i_role[i]+1: i_role[i+1]+1]
        else: L_i = team_i_11[i_role[i]+2: i_role[i+1]+1]
        if len(L_i) == 2: # played full time
            M_ppp[team_num].append(L_i + [1.0])
        else: # did not play full time
            M_ppp[team_num].append([L_i[0], L_i[2], int(L_i[1][:-1])/90.0])
    # iterate over substitute players
    i_role = [0] + [i for i in range(len(team_i_sub)) if team_i_sub[i] in role]
    for i in range(len(i_role)-1):
        if i == 0: L_i = team_i_sub[i_role[i]+1: i_role[i+1]+1]
        else: L_i = team_i_sub[i_role[i]+2: i_role[i+1]+1]
        if len(L_i) == 3: # sub player played
            M_ppp[team_num].append([L_i[0], L_i[2], (90.0-int(L_i[1][:-1]))/90.0])
H_ppp, A_ppp = np.array(M_ppp[0]), np.array(M_ppp[1])
print(H_ppp)
#print(A_ppp)

[['Mat Ryan' 'Goalkeeper' '1.0']
 ['Lewis Dunk' 'Defender' '1.0']
 ['Bruno' 'Defender' '0.9333333333333333']
 ['Bernardo' 'Defender' '1.0']
 ['Shane Duffy' 'Defender' '1.0']
 ['Yves Bissouma' 'Midfielder' '1.0']
 ['Pascal Groß' 'Midfielder' '1.0']
 ['Alireza Jahanbakhsh' 'Midfielder' '0.7444444444444445']
 ['Beram Kayal' 'Midfielder' '1.0']
 ['Anthony Knockaert' 'Midfielder' '1.0']
 ['Glenn Murray' 'Forward' '0.7444444444444445']
 ['Martín Montoya' 'Defender' '0.06666666666666667']
 ['Florin Andone' 'Forward' '0.25555555555555554']
 ['Jürgen Locadia' 'Forward' '0.25555555555555554']]


In [7]:

#print(soup_team)


In [8]:
import re
data_past_seasons=soup_team.find_all('option', string=re.compile("Premier"))
print(data_past_seasons)
tag_lister=[]
for option in data_past_seasons:
    tag_lister.append(option['value'])
print(tag_lister)


[<option value="16368">Premier League - 2018/2019</option>, <option value="15151">Premier League - 2017/2018</option>, <option value="13796">Premier League - 2016/2017</option>, <option value="12496">Premier League - 2015/2016</option>, <option value="9155">Premier League - 2014/2015</option>, <option value="7794">Premier League - 2013/2014</option>, <option value="6531">Premier League - 2012/2013</option>, <option value="5476">Premier League - 2011/2012</option>, <option value="4345">Premier League - 2010/2011</option>, <option value="3115">Premier League - 2009/2010</option>]
['16368', '15151', '13796', '12496', '9155', '7794', '6531', '5476', '4345', '3115']


In [9]:
for i in range(1):
    driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")
    url_base="https://www.whoscored.com/Teams/26/Archive/England-Liverpool?stageId="
    url_adder=tag_lister[i]
    historical_url=url_base+url_adder
    historical_text=driver.get(historical_url)
    soup_historical = BeautifulSoup(driver.page_source, 'lxml')
    historical_table=soup_historical.find_all('a', href=True)

    for player in historical_table:
        link=player['href']
        if link.startswith('#team'):
            link_stats=url_base+url_adder+link
            driver.get(link_stats)
            
            soup_stats=BeautifulSoup(driver.page_source, 'lxml')
            link_players=soup_stats.find_all('a',class_='player-link')
            stats_players=soup_stats.find_all('tr', class_=True)
            for playerstat in stats_players:
                print(playerstat.getText())
            
            print(link_stats)
        #print(player.getText())
        #print('\n')
        #print(type(player['href']))
    driver.quit()
        #print(player.getText())
        #print('\n')
        #print(type(player['href']))
   
    
    

	1  Mohamed Salah Liverpool, 27,  AM(CLR),FW   175 71 37(1)3262	22	8	1	-	3.6	75.9	0.3	8	7.57 
	2  Virgil van Dijk Liverpool, 28,  D(C)   193 92 383385	4	2	1	-	0.8	89.7	4.8	4	7.44 
	3  Sadio Mané Liverpool, 27,  AM(CLR),FW   175 69 35(1)3086	22	1	2	-	2.4	78.2	1	6	7.43 
	4  Trent Alexander-Arnold Liverpool, 21,  D(R),M(R)   175 69 27(2)2467	1	12	3	-	1	77.5	0.3	1	7.36 
	5  Roberto Firmino Liverpool, 28,  M(CLR),FW   181 76 31(3)2620	12	6	-	-	2.2	80.8	0.9	4	7.20 
	6  Andrew Robertson Liverpool, 25,  D(L),M(L)   178 64 363219	-	11	4	-	0.4	83.2	0.7	1	7.12 
	7  Joel Matip Liverpool, 28,  D(C)   195 90 17(5)1535	1	-	3	-	0.4	86.9	3	2	7.11 
	8  Dejan Lovren Liverpool, 30,  D(C)   188 84 11(2)986	1	-	1	-	0.5	86.6	4.1	1	7.08 
	9  Fabinho Liverpool, 25,  D(R),DMC   188 78 21(7)2013	1	2	6	-	0.6	85.3	2	-	6.95 
	10  James Milner Liverpool, 33,  D(LR),M(CLR)   175 70 19(12)1787	5	4	5	1	0.8	84.6	0.6	1	6.91 
	11  Georginio Wijnaldum Liverpool, 28,  M(CL)   175 69 32(3)2736	3	-	3	-	0.9	91.1	0.9	1	6.89 
	1

	1  Mohamed Salah Liverpool, 27,  AM(CLR),FW   175 71 37(1)3262	22	8	1	-	3.6	75.9	0.3	8	7.57 
	2  Virgil van Dijk Liverpool, 28,  D(C)   193 92 383385	4	2	1	-	0.8	89.7	4.8	4	7.44 
	3  Sadio Mané Liverpool, 27,  AM(CLR),FW   175 69 35(1)3086	22	1	2	-	2.4	78.2	1	6	7.43 
	4  Trent Alexander-Arnold Liverpool, 21,  D(R),M(R)   175 69 27(2)2467	1	12	3	-	1	77.5	0.3	1	7.36 
	5  Roberto Firmino Liverpool, 28,  M(CLR),FW   181 76 31(3)2620	12	6	-	-	2.2	80.8	0.9	4	7.20 
	6  Andrew Robertson Liverpool, 25,  D(L),M(L)   178 64 363219	-	11	4	-	0.4	83.2	0.7	1	7.12 
	7  Joel Matip Liverpool, 28,  D(C)   195 90 17(5)1535	1	-	3	-	0.4	86.9	3	2	7.11 
	8  Dejan Lovren Liverpool, 30,  D(C)   188 84 11(2)986	1	-	1	-	0.5	86.6	4.1	1	7.08 
	9  Fabinho Liverpool, 25,  D(R),DMC   188 78 21(7)2013	1	2	6	-	0.6	85.3	2	-	6.95 
	10  James Milner Liverpool, 33,  D(LR),M(CLR)   175 70 19(12)1787	5	4	5	1	0.8	84.6	0.6	1	6.91 
	11  Georginio Wijnaldum Liverpool, 28,  M(CL)   175 69 32(3)2736	3	-	3	-	0.9	91.1	0.9	1	6.89 
	1

In [ ]:
match_url='https://www.whoscored.com/Matches/318365'
driver = webdriver.Chrome(r"C:\Users\varun\OneDrive\Documents\classes\CS229\Final_Project\chromedriver.exe")
match_text=driver.get(match_url)
match_soup=BeautifulSoup(driver.page_source, 'lxml')
match_center_list=match_soup.find_all('a', href=True, text='Match Centre')
lineup_url= match_center_list['href']
print(lineup_url)
'''
lineup_text=driver.get(lineup_url)
lineup_soup=BeautifulSoup(driver.page_source, 'lxml')
players=lineup_soup.find_all('div', class_='Player')
print(players)
'''

